# Exp13: Control Text Generation with a locally running LLM

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import Accelerator

Configuration

In [3]:
MODEL="mistralai/Mistral-7B-Instruct-v0.2"
device = 'cuda'

Load the model and generate three sentences (as indicated by the end of sequence tokens) and print the longest sentence. This re-ranking will be based on the grammar classifiers later.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(MODEL, device_map="auto", torch_dtype=torch.float16, cache_dir="/mnt/qb/work/meurers/mpb672/cache")
tokenizer = AutoTokenizer.from_pretrained(MODEL, cache_dir="/mnt/qb/work/meurers/mpb672/cache")

In [61]:
prompt = """<s>[INST] Continue the writing with language on CEFR level C2. [/INST]"""
story = "I've been meaning to write for ages"

In [62]:
eos_chars = [".", "!", "?"]
max_len = 512

def generate_candidate(input_ids, max_token_sentence = 64):
    generated_tokens = torch.tensor([], device=device)
    with torch.no_grad():
        for _ in range(max_token_sentence):
            next_token_logits = model(torch.cat(input_ids, generated_tokens)).logits
            next_token_id = torch.argmax(next_token_logits[:, -1, :], dim=-1).item()

            next_token = tokenizer.decode(next_token_id)
            generated_tokens = torch.cat(generated_tokens, torch.tensor([[next_token_id]]).to('cuda'), dim=1)

            if any(eos_char in next_token for eos_char in eos_chars):
                break

    return tokenizer.decode(generated_tokens, skip_special_tokens=True)

while len(story) < max_len:
    inputs = tokenizer(prompt + story, return_tensors="pt").to(device)
    candidates = [generate_candidate(inputs.input_ids)]
    print(candidates)
    
    # TODO: do appropriate candidate selection
    story += candidates[0]


, but my schedule has been rather hectic lately.
, but my schedule has been rather hectic lately. Between juggling work commitments, family obligations, and social engagements, it's been a challenge to find the time and energy to put pen to paper, or rather, fingers to keyboard.
, but my schedule has been rather hectic lately. Between juggling work commitments, family obligations, and social engagements, it's been a challenge to find the time and energy to put pen to paper, or rather, fingers to keyboard. However, I've finally managed to carve out a few hours for myself, and I'm determined to make the most of it.
, but my schedule has been rather hectic lately. Between juggling work commitments, family obligations, and social engagements, it's been a challenge to find the time and energy to put pen to paper, or rather, fingers to keyboard. However, I've finally managed to carve out a few hours for myself, and I'm determined to make the most of it.

, but my schedule has been rather hec